# BB84 Quantum Key Distribution (QKD) Protocol using Qiskit

This notebook is a _demonstration_ of the BB84 Protocol for QKD using Qiskit. 
BB84 is a quantum key distribution scheme developed by Charles Bennett and Gilles Brassard in 1984 ([paper]).
The first three sections of the paper are readable and should give you all the necessary information required. 

![QKD Setup](https://raw.githubusercontent.com/deadbeatfour/quantum-computing-course/master/img/qkd.png)

[paper]: http://researcher.watson.ibm.com/researcher/files/us-bennetc/BB84highest.pdf 


In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, execute
from qiskit.providers.aer import QasmSimulator
from qiskit.visualization import *

## Choosing bases and encoding states

Alice generates two binary strings. One encodes the basis for each qubit:

$0 \rightarrow$ Computational basis

$1  \rightarrow$ Hadamard basis

The other encodes the state:

$0  \rightarrow|0\rangle$ or $|+\rangle $ 

$1  \rightarrow|1\rangle$  or  $|-\rangle $ 

Bob also generates a binary string and uses the same convention to choose a basis for measurement


In [89]:
num_qubits = 16

alice_basis = np.random.randint(2, size=num_qubits)
alice_state = np.random.randint(2, size=num_qubits)
bob_basis = np.random.randint(2, size=num_qubits)


print(f"Alice's State:\t {alice_state}")
print(f"Alice's Bases:\t {alice_basis}")
print(f"Bob's State:\t {bob_basis}")

Alice's State:	 [1 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0]
Alice's Bases:	 [1 0 1 0 1 1 0 0 1 0 1 0 1 1 0 0]
Bob's State:	 [0 1 0 1 0 1 1 0 1 1 1 0 0 1 1 0]


## Creating the circuit

Based on the following results:

$X|0\rangle = |1\rangle$

$H|0\rangle = |+\rangle$

$ HX|0\rangle = |-\rangle$

Our algorithm to construct the circuit is as follows:

1. Whenever Alice wants to encode 1 in a qubit, she applies an $X$ gate to the qubit. To encode 0, no action is needed.
2. Wherever she wants to encode it in the Hadamard basis, she applies an $H$ gate. No action is necessary to encode a qubit in the computational basis.

3. She then _sends_ the qubits to Bob (symbolically represented in this circuit using wires)

4. Bob measures the qubits according to his binary string. To measure a qubit in the Hadamard basis, he applies an $H$ gate to the corresponding qubit and then performs a mesurement on the computational basis. 



In [90]:
bb84_circ = QuantumCircuit(num_qubits)

# Alice prepares her qubits
for index in range(len(alice_basis)):
    if alice_state[index] == 1:
        bb84_circ.x(index)
    if alice_basis[index] == 1:
        bb84_circ.h(index)
bb84_circ.barrier()  

# Bob measures the received qubits
for index in range(len(bob_basis)):
    if bob_basis[index] == 1:
        bb84_circ.h(index)
        
bb84_circ.barrier()        
bb84_circ.measure_all()
    
            
# bb84_circ.draw('mpl')


## Performing the actual experiment

Now we will run this simulation experiment on the `QasmSimulator()` backend.

In [91]:
job = execute(bb84_circ.reverse_bits(),backend=QasmSimulator(),shots=10)
counts = job.result().get_counts()

# Prints out all the measured binary strings by Bob (reversed)
# for count in [*counts]:
#     print(count) 

## Creating the key

Alice and Bob only keep the bits where their bases match.

The following outcomes are possible for each bit sent using the BB84 protocol

|     Alice's   bit    	|     Alice's   basis    	|     Alice's   State    	|     Bob's   basis    	|     Bob's   outcome    	|     Bob's   bit    	|     Probability    	|
|----------------------	|------------------------	|------------------------	|----------------------	|------------------------	|--------------------	|--------------------	|
|     0                	|     C                  	|     0                  	|     C                	|     0                  	|     0              	|     1/8            	|
|     0                	|     C                  	|     0                  	|     H                	|     +                  	|     0              	|     1/16           	|
|     0                	|     C                  	|     0                  	|     H                	|     -                  	|     1              	|     1/16           	|
|     0                	|     H                  	|     +                  	|     C                	|     0                  	|     0              	|     1/16           	|
|     0                	|     H                  	|     +                  	|     C                	|     1                  	|     1              	|     1/16           	|
|     0                	|     H                  	|     +                  	|     H                	|     +                  	|     0              	|     1/8            	|
|     1                	|     C                  	|     1                  	|     C                	|     1                  	|     1              	|     1/8            	|
|     1                	|     C                  	|     1                  	|     H                	|     +                  	|     0              	|     1/16           	|
|     1                	|     C                  	|     1                  	|     H                	|     -                  	|     1              	|     1/16           	|
|     1                	|     H                  	|     -                  	|     C                	|     0                  	|     0              	|     1/16           	|
|     1                	|     H                  	|     -                  	|     C                	|     1                  	|     1              	|     1/16           	|
|     1                	|     H                  	|     -                  	|     H                	|     -                  	|     1              	|     1/8            	|

\begin{align*}
P_{\text{same basis}} &= P_A(C)\times P_B(C) + P_A(H)\times P_B(H)\\
&= \frac{1}{2} \times \frac{1}{2} + \frac{1}{2} \times \frac{1}{2} \\ 
&= \frac{1}{2}
\end{align*}

Thus, on average, only half of the total bits will be in the final key. It is also interesting to note that half of the key bits will be 0 and the other half will be 1 (again, on average)

In [92]:
counts = execute(bb84_circ.reverse_bits(),backend=QasmSimulator(),shots=10).result().get_counts()
keys = []
for temp_key in counts:
    key = ''
    for i in range(num_qubits):
        if alice_basis[i] == bob_basis[i]: # Only choose bits where Alice and Bob chose the same basis
             key += str(temp_key[i])
    keys.append(key)
#     print(key)
print(f'The length of the key is {len(keys[0])}')
print(f"The key contains {(keys[0]).count('0')} zeroes and {(keys[0]).count('1')} ones")

The length of the key is 7
The key contains 5 zeroes and 2 ones


In [93]:
for key in keys:
    print(key)

1000100
1000100
1000100
1000100
1000100
1000100
1000100
1000100
1000100
1000100
